In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
import evaluate
import torch

In [2]:
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA version used to compile PyTorch: {torch.version.cuda}")

PyTorch version: 2.8.0+cu128
CUDA version used to compile PyTorch: 12.8


In [3]:
model_name = "Falconsai/text_summarization"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Explicitly move to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(f"Using model {model_name} on {device}")
print(f"Model device: {next(model.parameters()).device}")

Using model Falconsai/text_summarization on cuda
Model device: cuda:0


In [4]:
# Use only a small subset for faster experimentation
raw_datasets = load_dataset("cnn_dailymail", "3.0.0")
# Take only first 1000 examples for train and 200 for validation
small_train = raw_datasets["train"].select(range(1000))
small_val = raw_datasets["validation"].select(range(200))
raw_datasets["train"] = small_train
raw_datasets["validation"] = small_val
raw_datasets["train"], raw_datasets["validation"], raw_datasets["test"]

(Dataset({
     features: ['article', 'highlights', 'id'],
     num_rows: 1000
 }),
 Dataset({
     features: ['article', 'highlights', 'id'],
     num_rows: 200
 }),
 Dataset({
     features: ['article', 'highlights', 'id'],
     num_rows: 11490
 }))

In [5]:
max_input_length = 256
max_target_length = 64

def preprocess_function(examples):
    inputs = examples["article"]
    # you can prefix with a task indicator (optional for some models)
    inputs = ["summarize: " + doc for doc in inputs]
    model_inputs = tokenizer(
        inputs,
        max_length=max_input_length,
        truncation=True,
        padding="max_length"
    )

    # tokenize summaries (targets)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["highlights"],
            max_length=max_target_length,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True, remove_columns=raw_datasets["train"].column_names)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

c:\Users\enisw\miniconda3\Lib\site-packages\transformers\tokenization_utils_base.py:4007: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

rouge = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # decode, skip special tokens
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Sometimes one summary is empty string after decoding → fix that
    decoded_preds = [pred if pred.strip() != "" else " " for pred in decoded_preds]
    decoded_labels = [label if label.strip() != "" else " " for label in decoded_labels]

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    
    # Fix: Handle different ROUGE metric formats
    if isinstance(result["rouge1"], dict) and "mid" in result["rouge1"]:
        # Old format with .mid attribute
        result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    else:
        # New format - direct float values
        result = {key: value * 100 for key, value in result.items()}
    
    # optional: add generation length
    prediction_lens = [len(tokenizer(p).input_ids) for p in decoded_preds]
    result["gen_len"] = sum(prediction_lens) / len(prediction_lens)
    return result

In [8]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./summarization_model",
    eval_strategy="epoch",
    per_device_train_batch_size=8,  # Start smaller if GPU memory is limited
    per_device_eval_batch_size=8,
    learning_rate=3e-5,
    weight_decay=0.01,
    save_strategy="epoch",
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    logging_dir="./logs",
    logging_steps=100,
    dataloader_num_workers=0,  # Set to 0 on Windows to avoid multiprocessing issues
    remove_unused_columns=False,
    no_cuda=False,  # Explicitly enable CUDA
)

In [9]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

C:\Users\enisw\AppData\Local\Temp\ipykernel_6720\1485828738.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [10]:
trainer.train()

Epoch,Training Loss,Validation Loss


AttributeError: 'numpy.float64' object has no attribute 'mid'